In [8]:
import bpy
import numpy as np

import os
from typing import List, Dict
from collections import deque
import bmesh

import scipy.sparse
import scipy.linalg
from scipy.sparse.linalg import spsolve
import matplotlib.pyplot as plt

from line_profiler import profile

from mathutils import Vector, Matrix

# install packages : "C:\Users\Pierre.Gilibert\OneDrive - ARVERNE\Bureau\blender-4.3.1-windows-x64\4.3\python\bin\python.exe" -m pip install scipy 

In [9]:
FOLDER = "C:\\Users\\pierr\\Documents\\Blender\\GeometryProcessing_Python\\"
FOLDER = "C:\\Users\\Pierre.Gilibert\\OneDrive - ARVERNE\\Documents\\Divers\\blender"
bpy.ops.wm.open_mainfile(filepath=os.path.join(FOLDER, "blender_notebook_v01.blend"))

{'FINISHED'}

In [10]:


class HEEdge:
    def __init__(self, edge, face, orientation:int):
        self.edge = edge
        self.face = face
        self.orientation = orientation
        self.vertex = edge.verts[orientation] # orientation in {0,1} stating whether the self.edge is in the same (0) direction as self or the opposite (1) direction.
        self.next = None
        self.twin = None
        self.angle_from_X = 0.0
        self.transport_coeff = 0.0

    def __str__(self):
        return f"HEedge : {self.edge} - orientation: {self.orientation}"
    
    def __repr__(self):
        return f"HEedge : {self.edge} - orientation: {self.orientation}"

    def __getattr__(self, item):
        # Delegate attribute access to the internal bmesh instance
        return getattr(self.edge, item)

    


In [11]:

class MYMesh:
    def __init__(self):
        self.bm = bmesh.new()
        self.facecorners = []
        self.vert2facecorner = {}
        self.facecorner_attributes = {}             # facecorners attributes
        self.vertex_attributes = {"u":{}, "v":{}}   # u,v : basis vector of the tangent plane at each vertex
        self.edge_attributes = {}
        self.face_attributes = {}
        self.face_vertex_to_facecorner = {}
        self.co = None                              # A (|V|, 3) np array where row i contains the x,y,z coordinates of the vertex indexed by i 
        self.cotan = None                           # A (3|F|,) np array where entry i is the cotangent of the angle of facecorner (loop) i
        self.face_areas = None                      # A (|F|,) np array where entry i is the area for face i
        self.internal_angles = None                 # A (3|F|,) np array where entry i is the angle of facecorner (loop) i
        self.fv = None                              # A (|F|, 3) np array where row i contains the indices of the vertices of face i
        self.heedges = []                           # List of Half edges
        self.dict_vert2heedges = {}

    def from_mesh(self, mesh_data):
        """ Mimic the bmesh from_mesh function. """
        self.bm.from_mesh(mesh_data)
        self.vert2facecorner = {v:[] for v in self.verts}
        self.facecorners = []
        for face in self.faces:
            self.face_vertex_to_facecorner[face] = {}
            for loop in face.loops:
                self.vert2facecorner[loop.vert].append(loop)
                self.facecorners.append(loop)
                self.face_vertex_to_facecorner[face][loop.vert] = loop
            
        self.fv = np.array([[fc.vert.index for fc in f.loops] for f in self.faces], dtype=int) # face #f has vertex [vi, vj, vk]
        self.co = np.array([v.co for v in self.verts])
        
    def to_mesh(self, mesh_data):
        """ Mimic the bmesh to_mesh function. """
        self.bm.to_mesh(mesh_data)
    
    # Add any other bmesh methods as needed:
    def free(self):
        """ Mimic the bmesh free function. """
        self.bm.free()

    # Add a custom method to directly access the internal bmesh:
    def __getattr__(self, item):
        # Delegate attribute access to the internal bmesh instance
        return getattr(self.bm, item)
    
    def ensure_lookup_tables(self):
        self.verts.ensure_lookup_table()
        self.edges.ensure_lookup_table()
        self.faces.ensure_lookup_table()

    def create_halfedge_datastructure(self):
        self.dict_vert2heedges = {}
        self.vertex_attributes["hedge"] = {}
        # self.edge_attributes["heedges"] = {}
        # for e in self.edges:
        #     self.edge_attributes["heedges"][e] = []
        for f in self.faces:
            if len(f.verts) != 3:
                raise ValueError("Not a triangular mesh : triangulate it beforehand !!")
            v_orientation = [f.verts[0].index, f.verts[1].index, f.verts[2].index]
            face_heedges = []
            for e in f.edges:
                # e0, e1, e2 = f.edges[0], f.edges[1], f.edges[2]
                v0, v1 = v_orientation.index(e.verts[0].index), v_orientation.index(e.verts[1].index)
                orientation = 1 - ((abs(v0-v1) == 1 and v0 < v1) or (abs(v0-v1) == 2 and v0 > v1)) # 0 if the edge matches the ccw convention of the face, 1 otherwise
                # print(e, v_orientation.index(e.verts[0].index), v_orientation.index(e.verts[1].index), orientation)
                face_heedges.append(HEEdge(e, f, orientation))
            he0, he1, he2 = face_heedges
            he0.next = he1
            he1.next = he2
            he2.next = he0
            self.dict_vert2heedges[(he0.vertex.index, he1.vertex.index)] = he0
            self.dict_vert2heedges[(he1.vertex.index, he2.vertex.index)] = he1
            self.dict_vert2heedges[(he2.vertex.index, he0.vertex.index)] = he2
            self.heedges.append(he0)
            self.heedges.append(he1)
            self.heedges.append(he2)
            self.vertex_attributes["hedge"][he0.vertex] = he0
            self.vertex_attributes["hedge"][he1.vertex] = he1
            self.vertex_attributes["hedge"][he2.vertex] = he2
            # self.edge_attributes["heedges"][he0.edge] = he0
            # self.edge_attributes["heedges"][he1.edge] = he1
            # self.edge_attributes["heedges"][he2.edge] = he2

        for k in self.dict_vert2heedges:
            i0, i1 = k
            if i0 < i1:
                continue
            kr = (i1, i0)
            if kr in self.dict_vert2heedges:
                self.dict_vert2heedges[k].twin = self.dict_vert2heedges[kr]
                self.dict_vert2heedges[kr].twin = self.dict_vert2heedges[k]
        
        boundary_hedges = []
        for e in self.edges:
            if not e.is_boundary:
                continue
            i0, i1 = e.verts[0].index, e.verts[1].index
            if (i0, i1) in self.dict_vert2heedges:
                existing_he = self.dict_vert2heedges[(i0, i1)]
            elif (i1, i0) in self.dict_vert2heedges:
                existing_he = self.dict_vert2heedges[(i1, i0)]
            else:
                raise ValueError(f"Unable to find an half edge between vertices {i0} and {i1}")
            new_he = HEEdge(e, None, 1-existing_he.orientation)
            existing_he.twin = new_he
            new_he.twin = existing_he
            boundary_hedges.append(new_he)
            self.dict_vert2heedges[(new_he.vertex.index, e.other_vert(new_he.vertex).index)] = new_he
            self.heedges.append(new_he)

        # match the boundary hedges
        num_link_to_create = len(boundary_hedges)
        num_link_created = 0
        while num_link_created != num_link_to_create:
            he = boundary_hedges[num_link_created]
            # other_index = -1
            for i, ohe in enumerate(boundary_hedges):
                if ohe.vertex == he.twin.vertex:
                    other_index = i
                    break
            # ohe = boundary_hedges.pop(other_index)
            he.next = ohe
            num_link_created+=1


        # verification : all hedges should have a twin and a next defined
        for k in self.dict_vert2heedges:
            he = self.dict_vert2heedges[k]
            if he.twin is None or he.next is None:
                raise ValueError(f"Unable to define a twin or a next for hedge {k} : {he}")

    def _build_d0(self):

        row = np.repeat(np.arange(len(self.edges)), 2)
        col = np.array([y for e in self.edges for y in [e.verts[0].index, e.verts[1].index]])
        val = np.ones(len(col))
        val[1::2] = -1
        # row, col, val = [], [], []
        # for e in self.edges:
        #     row.append(e.index)
        #     row.append(e.index)
        #     vi, vj = e.verts[0].index, e.verts[1].index
        #     col.append(vi)
        #     col.append(vj)
        #     if vi < vj:
        #         val.append(-1)
        #         val.append(1)
        #     else:
        #         val.append(1)
        #         val.append(-1)

        # matrix[e] = [0,0,..., 1, 0, ..., -1, ..., 0] +1 at vertex vi, -1 at vertex vj where e = [vi, vj] in that order
        return scipy.sparse.coo_matrix((val, (row, col)), shape=(len(self.edges), len(self.verts))).tocsr()

    def _build_d1(self):
        edge_dict = {tuple(sorted([e.verts[0].index, e.verts[1].index])): i for i, e in enumerate(self.edges)}
        vi, vj, vk = self.fv[:,0], self.fv[:,1], self.fv[:,2]
        face_edges = np.reshape(np.vstack((vi, vj, vj, vk, vk, vi)).T, (len(self.faces), 3, 2))
        # face_edges[f] = [[i,j], [j,k], [k, i]] (shape 3,2) with i,j,k the vertex indices of face f and [i,j], [j,k] and [k,i] the 3 ordered edges of face f
        sorted_face_edges = np.sort(face_edges, axis=2)
        val = np.any(sorted_face_edges == face_edges, axis=2)*2-1 # +1 if edge in correct orientation, -1 otherwise
        col = np.array([edge_dict[tuple(e)] for e in sorted_face_edges.reshape((-1,2))])
        row = np.repeat(np.arange(len(self.faces)), 3)
        # matrix[f] = [0, 0,... , +-1, ..., +-1, ..., +-1, ..., 0] non 0 at ei, ej, ek where f.edges = [ei, ej, ek] and +-1 depending on whether the vertices of the ei, ej, ek are in the sorted order or not 
        return scipy.sparse.coo_matrix((val.ravel(), (row, col)), shape=(len(self.faces), len(self.edges))).tocsr()

    def _build_hodge0(self, inverse=False):
        if not "area" in self.vertex_attributes:
            self._calculate_vertex_area()
        vertex_areas = self.vertex_attributes["area"]
        N = len(self.verts)
        row = np.arange(N)
        if inverse:
            return scipy.sparse.coo_matrix((1/vertex_areas, (row, row)), shape=(N, N))
        else:
            return scipy.sparse.coo_matrix((vertex_areas, (row, row)), shape=(N, N)) # dual / primal with primal = area of vertex = 1 by convention

    def _build_hodge1(self, inverse=False):
        edge_vertex = np.array([[e.verts[0].index, e.verts[1].index] for e in self.edges])
        edge_face_areas = np.zeros(len(self.edges))
        for e in self.edges:
            s = 0
            for f in e.link_faces:
                s += self.face_areas[f.index]
            edge_face_areas[e.index] = s

        edge_len = np.linalg.norm(self.co[edge_vertex[:,1]] - self.co[edge_vertex[:,0]], axis=1)
        # dual_edge_lengths = edge_face_areas / edge_len

        N = len(self.edges)
        row = np.arange(N)
        # return edge_len / edge_face_areas
        if inverse:
            return scipy.sparse.coo_matrix((edge_len / edge_face_areas, (row, row)), shape=(N, N))
        else:
            return scipy.sparse.coo_matrix((edge_face_areas / edge_len, (row, row)), shape=(N, N))

    def _build_hodge2(self, inverse=False):
        if not "area" in self.vertex_attributes:
            self._calculate_vertex_area()

        N = len(self.faces)
        row = np.arange(N)
        primal_areas = self.face_areas

        dual_areas = 1 #

        # fc_area = np.array([[self.facecorner_attributes["area"][self.face_vertex_to_facecorner[f][v].index] for f in self.faces for v in f.verts]]).reshape((-1,3))
        # weights = fc_area / self.face_areas[:,None]
        # vertex_area = self.vertex_attributes["area"][self.fv.ravel()].reshape((-1,3))
        # dual_areas = weights*vertex_area
        # dual_areas = dual_areas[:,0] + dual_areas[:,1] + dual_areas[:,2]
        if inverse:
            return scipy.sparse.coo_matrix((primal_areas/dual_areas, (row, row)), shape=(N, N))
        else:
            return scipy.sparse.coo_matrix((dual_areas/primal_areas, (row, row)), shape=(N, N))

    def construct_dec_operators(self, inverse_hodge0=False, inverse_hodge1=False, inverse_hodge2=False):
        """
        Constructs discrete exterior derivative operators (d0, d1) and
        Hodge star matrices (hodge0, hodge1, hodge2).
        """
        d0 = self._build_d0()
        d1 = self._build_d1()
        hodge0 = self._build_hodge0(inverse=inverse_hodge0)
        hodge1 = self._build_hodge1(inverse=inverse_hodge1)
        hodge2 = self._build_hodge2(inverse=inverse_hodge2)
        
        return d0, d1, hodge0, hodge1, hodge2

    def vector_field_to_1form(self, vector_field):
        """
        Projects a vertex-based tangent vector field onto a discrete 1-form on edges.

        Parameters:
        - vector_field: Nx3 numpy array, the tangent vector field at each vertex.

        Returns:
        - one_form: Mx1 numpy array, the discrete 1-form (edge-based representation).
        """
        
        edge_vertex = np.array([[e.verts[0].index, e.verts[1].index] for e in self.edges])
        edge_dir = self.co[edge_vertex[:,1]] - self.co[edge_vertex[:,0]]
        ui = vector_field[edge_vertex[:,0]]
        uj = vector_field[edge_vertex[:,1]]
        u_edge = 0.5 * (ui+uj)
        dot = np.einsum('ij,ij->i', u_edge, edge_dir) # fast dot product 
        return dot/np.linalg.norm(edge_dir, axis=1)
    
    def one_form_to_vector_field(self, one_form):
        """
        Maps a discrete 1-form (edge-based representation) back to a vertex-based tangent vector field.

        Parameters:
        - one_form: Mx1 numpy array, the discrete 1-form values on edges.

        Returns:
        - vector_field: Nx3 numpy array, the tangent vector field at each vertex.
        """

        edge_vertex = np.array([[e.verts[0].index, e.verts[1].index] for e in self.edges])
        edge_dir = self.co[edge_vertex[:,1]] - self.co[edge_vertex[:,0]]
        edge_dir = edge_dir / np.linalg.norm(edge_dir, axis=1)[:,None]
        contribution = one_form[:,None] * edge_dir

        row = np.repeat(np.arange(len(self.edges)),2).astype(int)
        col = edge_vertex.ravel().astype(int)
        val = np.ones(2 * len(self.edges))

        adjacency_matrix = scipy.sparse.coo_matrix((val, (row, col)))

        vertex_contributions = adjacency_matrix.T @ contribution  # Sum contributions per vertex
        vertex_edge_counts = adjacency_matrix.sum(axis=0).A1  # Number of edges per vertex (1D array)

        return vertex_contributions / vertex_edge_counts[:,None]

    def _calculate_corner_area(self):
        if self.internal_angles is None:
            self._calculate_corner_angles_and_face_areas()

        nfaces = len(self.faces)
        ffc = np.arange(3*nfaces).reshape((nfaces, 3)) # face corner index : face #f has corners [i, j, k];
        angles = self.internal_angles[ffc] # array of alpha_i, alpha_j, alpha_k
        eij = self.co[self.fv[:,1]] - self.co[self.fv[:,0]] # edges
        ejk = self.co[self.fv[:,2]] - self.co[self.fv[:,1]]
        eki = self.co[self.fv[:,0]] - self.co[self.fv[:,2]]
        lij2 = eij[:,0]*eij[:,0] + eij[:,1]*eij[:,1] + eij[:,2]*eij[:,2] # squared lengths of the edges
        ljk2 = ejk[:,0]*ejk[:,0] + ejk[:,1]*ejk[:,1] + ejk[:,2]*ejk[:,2]
        lki2 = eki[:,0]*eki[:,0] + eki[:,1]*eki[:,1] + eki[:,2]*eki[:,2]

        # in case triangle i,j,k is non obtuse (all its angles are < pi/2) then this is the area of each face corner
        non_obtuse_area_i = 0.125 * (lij2 * self.cotan[ffc[:,2]] + lki2 * self.cotan[ffc[:,1]])
        non_obtuse_area_j = 0.125 * (ljk2 * self.cotan[ffc[:,0]] + lij2 * self.cotan[ffc[:,2]])
        non_obtuse_area_k = 0.125 * (lki2 * self.cotan[ffc[:,1]] + ljk2 * self.cotan[ffc[:,0]])

        facecorners_areas = np.zeros(3*nfaces).reshape((nfaces, 3))
        # check whether the angles are less than pi/2
        small_angle_bool = angles < np.pi/2
        big_angle_bool = ~small_angle_bool
        # True for all corners of triangle i,j,k ==> use the non_obtuse_area
        non_obtuse_bool = np.logical_and(small_angle_bool[:,0], np.logical_and(small_angle_bool[:,1], small_angle_bool[:,2]))
        facecorners_areas[non_obtuse_bool, 0] = non_obtuse_area_i[non_obtuse_bool]
        facecorners_areas[non_obtuse_bool, 1] = non_obtuse_area_j[non_obtuse_bool]
        facecorners_areas[non_obtuse_bool, 2] = non_obtuse_area_k[non_obtuse_bool]
        # False for corner p in {i,j,k}: use half the face area for p and a quarter for the two others corners
        facecorners_areas[big_angle_bool[:,0]] = self.face_areas[big_angle_bool[:,0]][:,None] * np.array([[0.5, 0.25, 0.25]])
        facecorners_areas[big_angle_bool[:,1]] = self.face_areas[big_angle_bool[:,1]][:,None] * np.array([[0.25, 0.5, 0.25]])
        facecorners_areas[big_angle_bool[:,2]] = self.face_areas[big_angle_bool[:,2]][:,None] * np.array([[0.25, 0.25, 0.5]])

        self.facecorner_attributes["area"] = facecorners_areas.ravel()

    def _calculate_corner_angles_and_face_areas(self):

        vi = self.co[self.fv[:,0]]
        vj = self.co[self.fv[:,1]]
        vk = self.co[self.fv[:,2]]

        eij, ejk, eki = vj-vi, vk-vj, vi-vk
        lij2 = eij[:,0]*eij[:,0] + eij[:,1]*eij[:,1] + eij[:,2]*eij[:,2]
        ljk2 = ejk[:,0]*ejk[:,0] + ejk[:,1]*ejk[:,1] + ejk[:,2]*ejk[:,2]
        lki2 = eki[:,0]*eki[:,0] + eki[:,1]*eki[:,1] + eki[:,2]*eki[:,2]
        lij, ljk, lki = np.sqrt(lij2), np.sqrt(ljk2), np.sqrt(lki2)

        s = 0.5 * (lij + ljk + lki) # half perimeter of every triangle
        self.face_areas = np.sqrt(s * (s - lij) * (s - ljk) * (s - lki)) # Heron's formula for the area of the triangles
        
        q_i = -ljk2 + lij2 + lki2
        q_j = -lki2 + ljk2 + lij2
        q_k = -lij2 + lki2 + ljk2

        denom_inv = 1/(4*self.face_areas)
        self.cotan = np.zeros(3*len(bm.faces))
        self.cotan[0::3] = q_i*denom_inv
        self.cotan[1::3] = q_j*denom_inv
        self.cotan[2::3] = q_k*denom_inv

        self.internal_angles = np.zeros(3*len(bm.faces))
        self.internal_angles[0::3] = np.arccos(np.clip(q_i / (2*lij * lki), -1, 1))
        self.internal_angles[1::3] = np.arccos(np.clip(q_j / (2*ljk * lij), -1, 1))
        self.internal_angles[2::3] = np.arccos(np.clip(q_k / (2*lki * ljk), -1, 1))

    def _calculate_vertex_area(self, force_recompute=False):
        # self.vertex_attributes["area"] = np.array([np.sum([self.facecorner_attributes["area"][fc.index] for fc in self.vert2facecorner[v]]) for v in self.verts])
        if not "area" in self.facecorner_attributes or force_recompute:
            self._calculate_corner_area()
            
        val = []
        for v in self.verts:
            s = 0
            for fc in self.vert2facecorner[v]:
                s += self.facecorner_attributes["area"][fc.index]
            val.append(s)
        self.vertex_attributes["area"] = np.array(val)
      
    def assign_distinguished_vector_X(self):
        all_X = []
        for v in self.verts:
            if v.is_boundary:
                boundary_edges = [e for e in v.link_edges if e.is_boundary]
                if len(boundary_edges) != 2:
                    raise ValueError(f"Non manifold mesh : vertex {v.index} does not have exactly 2 adjacent boundary edges")
                other_vs = [boundary_edges[0].other_vert(v), boundary_edges[1].other_vert(v)]
                # select the hedge starting at v and having a non None face
                vector_X = None
                for ov in other_vs:
                    key = (v.index, ov.index)
                    key_r = (ov.index, v.index)
                    for k in [key, key_r]:
                        he = self.dict_vert2heedges[k]
                        if he.vertex == v and he.face is not None:
                            vector_X = he
                            break
                    if vector_X is not None:
                        break
                if vector_X is None:
                    raise ValueError(f"Unable to find an X for boundary vertex {v.index}")
                all_X.append(vector_X)
                # if (v.index, boundary_edges[0].other_vert(v).index) in self.dict_vert2heedges:
                #     all_X.append(self.dict_vert2heedges[(v.index, boundary_edges[0].other_vert(v).index)])
                # elif (v.index, boundary_edges[1].other_vert(v).index) in self.dict_vert2heedges:
                #     all_X.append(self.dict_vert2heedges[(v.index, boundary_edges[1].other_vert(v).index)])
                # else:
                #     raise ValueError(f"Unable to find an X for boundary vertex {v.index}")
            else:
                all_X.append(self.dict_vert2heedges[(v.index, v.link_edges[0].other_vert(v).index)])
        # all_X = np.array(all_X)
        self.vertex_attributes["X"] = all_X 

    def calculate_angle_sum(self, force_recompute=False):
        if self.internal_angles is None or force_recompute:
            self._calculate_corner_angles_and_face_areas()
        angle_sum = np.zeros(len(self.verts))
        for i, v in enumerate(self.verts):
            s = 0
            for fc in self.vert2facecorner[v]:
                s += self.internal_angles[fc.index]
            angle_sum[i] = s

        self.vertex_attributes["angle_sum"] = angle_sum

    def calculate_angle_defect(self, force_recompute=False):
        if not "angle_sum" in self.vertex_attributes or force_recompute:
            self.calculate_angle_sum(force_recompute)

        boundary_coeff =  2 - np.array([v.is_boundary for v in self.verts]) # v on interior : 2 ; v on boundary : 1
        self.vertex_attributes["angle_defect"] = np.pi * boundary_coeff - self.vertex_attributes["angle_sum"]

    def calculate_gaussian_curvature(self, force_recompute=False):
        if not "area" in self.vertex_attributes or force_recompute:
            self._calculate_vertex_area(force_recompute)
        if not "angle_defect" in self.vertex_attributes or force_recompute:
            self.calculate_angle_defect(force_recompute)

        self.vertex_attributes["gaussian_curvature"] = self.vertex_attributes["angle_defect"] / self.vertex_attributes["area"]

    def compute_face_frame(self):

        vi, vj, vk = self.co[self.fv[:,0]], self.co[self.fv[:,1]], self.co[self.fv[:,2]]
        e1 = vj - vi
        e1 = e1 / np.linalg.norm(e1, axis=1)[:,None]
        e2 = vk - vi
        e2 = e2 - e1 * np.einsum('ij,ij->i',e1, e2)[:,None]
        e2 = e2/np.linalg.norm(e2, axis=1)[:,None]

        self.face_attributes["e1"] = e1
        self.face_attributes["e2"] = e2





In [13]:
obj = bpy.data.objects["Sphere"]

bpy.ops.object.mode_set(mode='OBJECT')
mesh = obj.data
bm = MYMesh()
bm.from_mesh(mesh)
bm.ensure_lookup_tables()
bm.create_halfedge_datastructure()




In [14]:
def build_d0d1(self:MYMesh):
    e2v = []
    f2e = np.zeros((len(self.faces), 3))
    edge_index = {}
    n_edges = 0

    for i, f in enumerate(self.faces):
        for j in range(3):
            v1 = f.verts[j].index
            v2 = f.verts[(j+1)%3].index

            if v2 < v1:
                v1, v2 = v2, v1
                sgn = -1
            else:
                sgn = 1

            if (v1, v2) not in edge_index:
                edge_index[(v1, v2)] = n_edges
                e2v.append([v1, v2])
                n_edges += 1

            f2e[i, j] = sgn * edge_index[(v1, v2)]
    e2v = np.array(e2v)
    # Build the exterior derivative matrices
    n_vertices = len(self.verts)

    # d0 matrix
    rows_d0 = []
    cols_d0 = []
    data_d0 = []
    for i in range(e2v.shape[0]):
        rows_d0.extend([i, i])
        cols_d0.extend([e2v[i, 0], e2v[i, 1]])
        data_d0.extend([-1, 1])

    d0 = scipy.sparse.coo_matrix((data_d0, (rows_d0, cols_d0)), shape=(e2v.shape[0], n_vertices))

    # d1 matrix
    rows_d1 = []
    cols_d1 = []
    data_d1 = []
    for i in range(f2e.shape[0]):
        for j in range(3):
            rows_d1.append(i)
            cols_d1.append(abs(f2e[i, j]))  # Convert back to 0-based indexing
            data_d1.append(np.sign(f2e[i, j]))

    d1 = scipy.sparse.coo_matrix((data_d1, (rows_d1, cols_d1)), shape=(f2e.shape[0], e2v.shape[0]))
    return d0, d1
d00, d11 = build_d0d1(bm)
d0 = bm._build_d0()
d1 = bm._build_d1()
d00, d11, d0, d1 = d00.todense(), d11.todense(), d0.todense(), d1.todense()

In [15]:
def calculate_holonomy(self:MYMesh, vertex_index):
    self.compute_face_frame()
    he = self.vertex_attributes["hedge"][self.verts[vertex_index]]
    keep_looping = True
    sum = 0
    while keep_looping:
        faceI, faceJ = he.face, he.twin.face
        if faceI is None or faceJ is None:
            continue
        u, v = he.vertex, he.twin.vertex
        e = v.co - u.co
        if u.index > v.index:
            e = -e
        ei1, ei2 = self.face_attributes["e1"][faceI.index], self.face_attributes["e2"][faceI.index]
        ej1, ej2 = self.face_attributes["e1"][faceJ.index], self.face_attributes["e2"][faceJ.index]
        delta_ij = np.arctan2(np.dot(ei2, e), np.dot(ei1, e))
        delta_ji = np.arctan2(np.dot(ej2, e), np.dot(ej1, e))
        sum += delta_ji - delta_ij
        # print(u.index, v.index, delta_ji, delta_ij)
        he = he.twin.next
        keep_looping = he != self.vertex_attributes["hedge"][self.verts[vertex_index]]
    while sum >= np.pi : sum-=2*np.pi
    while sum < -np.pi : sum+=2*np.pi
    return sum


calculate_holonomy(bm, 4)


-0.00940767719559954

In [16]:

def ensure_helpers_collection():
    """
    Ensure that a collection named "Helpers" exists in the scene.
    Returns the "Helpers" collection.
    """
    if "Helpers" not in bpy.data.collections:
        helpers_collection = bpy.data.collections.new("Helpers")
        bpy.context.scene.collection.children.link(helpers_collection)
    return bpy.data.collections["Helpers"]


def draw_cylinder(pstart, pend, radius, name="Helper_Cylinder"):
    """
    Draw a cylinder using Blender's bmesh module.

    Parameters:
    pstart (Vector): The center of the bottom circular face.
    pend (Vector): A point on the center of the top circular face.
    radius (float): The radius of the cylinder.

    Returns:
    The Blender object representing the cylinder.
    """
    # Delete all objects named "Cylinder" in the scene
    for obj in bpy.data.objects:
        if obj.name == name:
            bpy.data.objects.remove(obj, do_unlink=True)
    # Calculate the cylinder's length (height) and direction vector
    direction = pend - pstart
    height = direction.length

    # Normalize the direction vector
    direction.normalize()

    # Create a new mesh and object for the cylinder
    mesh = bpy.data.meshes.new(name)
    obj = bpy.data.objects.new(name, mesh)
    # bpy.context.collection.objects.link(obj)
    # Add the object to the "Helpers" collection
    helpers_collection = ensure_helpers_collection()
    helpers_collection.objects.link(obj)

    # Create a bmesh object to define geometry
    bm = bmesh.new()
    bmesh.ops.create_cone(
        bm,
        cap_ends=True,
        segments=16,
        radius1=radius,
        radius2=radius/4,
        depth=height
    )

    # Apply a transformation matrix to orient the cylinder
    # Align the cylinder along the specified axis
    up = Vector((0, 0, 1))
    rot_axis = up.cross(direction)
    if rot_axis.length > 0:  # Ensure the rotation axis is valid
        rot_axis.normalize()
        angle = up.angle(direction)
        rot_matrix = Matrix.Rotation(angle, 4, rot_axis)

        bmesh.ops.rotate(bm, verts=bm.verts, cent=(0, 0, 0), matrix=rot_matrix)

    # Translate the cylinder to the starting position
    translation = bmesh.ops.translate(bm, verts=bm.verts, vec=(pstart+pend)/2)

    # Write the bmesh data to the mesh
    bm.to_mesh(mesh)
    bm.free()


def draw_sphere(centre, radius, name):
    """
    Draw a sphere using Blender's bmesh module.

    Parameters:
    centre (Vector): The center of the sphere.
    radius (float): The radius of the sphere.

    Returns:
    The Blender object representing the sphere.
    """
    # Delete all objects named "Sphere" in the scene
    for obj in bpy.data.objects:
        if obj.name == name:
            bpy.data.objects.remove(obj, do_unlink=True)

    # Create a new mesh and object for the sphere
    mesh = bpy.data.meshes.new(name)
    obj = bpy.data.objects.new(name, mesh)
    # bpy.context.collection.objects.link(obj)
    # Add the object to the "Helpers" collection
    helpers_collection = ensure_helpers_collection()
    helpers_collection.objects.link(obj)

    # Create a bmesh object to define geometry
    bm = bmesh.new()
    bmesh.ops.create_uvsphere(
        bm,
        u_segments=4,
        v_segments=4,
        radius= radius
    )

    # Translate the sphere to the specified center
    bmesh.ops.translate(bm, verts=bm.verts, vec=centre)

    # Write the bmesh data to the mesh
    bm.to_mesh(mesh)
    bm.free()

    return obj

In [ ]:
class TrivialConnection():
    def __init__(self, mesh:MYMesh):
        self.mesh = mesh
        self.singularities = None

    def read_singularities(self):
        # self.singularities = np.array([[0, 2]]) #one singularity on vertex 0 of index +2
        self.singularities = np.array([[0, 1], [300, 1]]) # two singularities on vertices 0, 300 of index +1
        # # self.singularities = np.array([[0, 1], [300, -1], [500, 2]]) # three singularities on vertices 0, 300, 500 of index +1, -1, +2
        # self.singularities = np.array([[0, 0.5], [300, -.5], [1500, 2]]) # three singularities on vertices 0, 300, 500 of index +.5, -.5, +2

    def compute_trivial_connection(self, d0, d1, K):
        A = d0.T
        b = K.copy()
        for row in self.singularities:
            vertex_index, singularity_index = row
            vertex_index = int(vertex_index)
            b[vertex_index] = K[vertex_index] - 2 * np.pi * singularity_index

        # x = scipy.sparse.linalg.lsqr(A, -b)
        # x = x[0] # retrieve the solution
        # # project to the nullspace 
        # y = spsolve(d1 @ d1.T, d1 @ x)
        # x = x - d1.T @ y

        AAT = A @ A.T
        y = spsolve(AAT, -b)
        x = A.T @ y
        print(f"max holonomy error = {np.max(np.abs(A@x+b))}")

        return x


    def parallel_transport(self, w0, faceI, faceJ, he:HEEdge, x):
        """
        Transport unit vector w0 frome faceI to faceJ separated by halfe edge he using the connection specified by x
        """

        ei1, ei2 = self.mesh.face_attributes["e1"][faceI.index], self.mesh.face_attributes["e2"][faceI.index]
        ej1, ej2 = self.mesh.face_attributes["e1"][faceJ.index], self.mesh.face_attributes["e2"][faceJ.index]
        ei3, ej3 = np.cross(ei1, ei2), np.cross(ej1, ej2)
        EI = np.vstack((ei1, ei2, ei3)).T
        EJ = np.vstack((ej1, ej2, ej3)).T

        k = he.edge.index
        u, v = he.vertex, he.twin.vertex
        e = v.co - u.co
        if u.index > v.index:
            e = -e
        delta_ij = np.arctan2(np.dot(ei2, e), np.dot(ei1, e))
        delta_ji = np.arctan2(np.dot(ej2, e), np.dot(ej1, e))

        angle = x[k] - delta_ij + delta_ji
        # angle = 0 - delta_ij + delta_ji

        c, s = np.cos(angle), np.sin(angle)
        R = np.array([[c, -s, 0], [s, c, 0], [0, 0, 1]])
        w = EJ @ R @ EI.T @ w0
        return w
    
    def parallel_transport_BIS(self, phi:float, he:HEEdge):

        u, v = he.vertex, he.twin.vertex
        e = v.co - u.co
        if u.index > v.index:
            e = -e
        faceI, faceJ = he.face, he.twin.face
        ei1, ei2 = self.mesh.face_attributes["e1"][faceI.index], self.mesh.face_attributes["e2"][faceI.index]
        ej1, ej2 = self.mesh.face_attributes["e1"][faceJ.index], self.mesh.face_attributes["e2"][faceJ.index]
        delta_ij = np.arctan2(np.dot(ei2, e), np.dot(ei1, e))
        delta_ji = np.arctan2(np.dot(ej2, e), np.dot(ej1, e))

        return phi - delta_ij + delta_ji


    def extract_vector_field(self, x):

        nfaces = len(self.mesh.faces)
        vf = np.zeros((nfaces, 3))
        has_been_visited = np.zeros(nfaces, dtype=bool)
        
        root = self.mesh.faces[0] # arbitrary face
        # arbitrary vector
        p0, p1 = root.verts[0].co, root.verts[1].co
        v0 = p1 - p0
        v0 = v0 / np.linalg.norm(v0)

        vf[root.index] = v0
        has_been_visited[root.index] = True
        queue = deque()
        queue.append(root)
        he:HEEdge
        
        
        while len(queue)>0:
            f = queue.popleft()

            vi, vj = f.verts[0].index, f.verts[1].index
            he = self.mesh.dict_vert2heedges[(vi, vj)]
            keep_looping = True
            v0 = vf[f.index]
            while keep_looping:
                g = he.twin.face
                if g is not None and not has_been_visited[g.index]:
                    vg = self.parallel_transport(vf[f.index], f, g, he, x)
                    vf[g.index] = vg
                    has_been_visited[g.index] = True
                    queue.append(g)

                    # angle = self.parallel_transport_BIS(x[he.edge.index], he)

                he = he.next
                keep_looping = he != self.mesh.dict_vert2heedges[(vi,vj)]

        return vf
            
    def compute_vector_field(self):
        self.read_singularities()
        self.mesh.compute_face_frame()

        d0 = self.mesh._build_d0()
        d1 = self.mesh._build_d1()
        # d0, d1 = build_d0d1(self.mesh)
        self.mesh.calculate_gaussian_curvature(force_recompute=True)
        K = self.mesh.vertex_attributes["angle_defect"]
        # K = -np.array([calculate_holonomy(self.mesh, i) for i in range(len(self.mesh.verts))])
        
        x = self.compute_trivial_connection(d0, d1, K)
        # print(x.shape)
        # print(len(self.mesh.edges))
        # vector_field = self.extract_vector_field(x)
        vector_field = self.extract_vector_fieldBIS(x)
        # vector_field = self.extract_vector_fieldTER(x)
        return vector_field
    

    def extract_vector_fieldTER(self, x):
        root = self.mesh.faces[0]
        nfaces = len(self.mesh.faces)
        alphas = np.zeros(nfaces)
        vector_field = np.zeros((nfaces, 3))
        has_been_visited = np.zeros(nfaces, dtype=bool)

        p0, p1 = root.verts[0].co, root.verts[1].co
        v0 = p1 - p0
        v0 = v0 / np.linalg.norm(v0)

        has_been_visited[root.index] = True
        vector_field[root.index] = v0


        queue = deque()
        queue.append(root)
        he:HEEdge
        K = self.mesh.vertex_attributes["angle_defect"]
        counter = 0
        while len(queue)>0:
            f = queue.popleft()
            vi, vj = f.verts[0].index, f.verts[1].index
            he = self.mesh.dict_vert2heedges[(vi, vj)]
            keep_looping = True
            while keep_looping:
                faceI, faceJ = he.face, he.twin.face
                if not has_been_visited[he.twin.face.index]:
                    u, v = he.vertex, he.twin.vertex
                    e = v.co - u.co
                    sgn = 1
                    # if u.index > v.index:
                    if he.edge.verts[0].index > he.edge.verts[1].index:
                        e = -e
                        sgn = -1
                    ei1, ei2 = self.mesh.face_attributes["e1"][faceI.index], self.mesh.face_attributes["e2"][faceI.index]
                    ej1, ej2 = self.mesh.face_attributes["e1"][faceJ.index], self.mesh.face_attributes["e2"][faceJ.index]
                    delta_ij = np.arctan2(np.dot(ei2, e), np.dot(ei1, e))
                    delta_ji = np.arctan2(np.dot(ej2, e), np.dot(ej1, e))
                    alpha_i = alphas[faceI.index]
                    alpha_j = alpha_i - delta_ij + delta_ji - sgn * x[he.edge.index]
                    alphas[faceJ.index] = alpha_j
                    vector_field[faceJ.index] = rotate_axis_angle_single_vector(ej1, faceJ.normal, alpha_j)
                    has_been_visited[faceJ.index] = True
                    queue.append(faceJ)
                he = he.next
                keep_looping = he != self.mesh.dict_vert2heedges[(vi, vj)]
            counter += 1
            if counter == 200:
                break
        return vector_field

    def draw_singularities(self):
        for row in self.singularities:
            vertex_index = row[0]
            co = self.mesh.verts[vertex_index].co
            draw_sphere(co, 0.1, f"singularity_{vertex_index}")

    def extract_vector_fieldBIS(self, x):
        root = self.mesh.faces[0]
        initial_angle = 0
        nfaces = len(self.mesh.faces)
        alphas = np.zeros(nfaces)
        alphas[root.index] = initial_angle

        vector_field = np.zeros((nfaces, 3))
        has_been_visited = np.zeros(nfaces, dtype=bool)


        p0, p1 = root.verts[0].co, root.verts[1].co
        # draw_sphere(p0, 0.01, "p0")
        # draw_sphere(p1, 0.01, "p1")
        # draw_cylinder(p0, p1, 0.005, name="p0p1Cylinder")
        v0 = p1 - p0
        v0 = v0/ np.linalg.norm(v0)

        v0 = rotate_axis_angle_single_vector(v0, root.normal, initial_angle)
        vector_field[root.index] = v0
        # has_been_visited[root.index] = True
        queue = deque()
        queue.append(root)
        he:HEEdge
        K = self.mesh.vertex_attributes["angle_defect"]
        A = self.mesh._build_d0().T
        Ad = A.todense()
        y = A @ x
        b = K.copy()
        for row in self.singularities:
            vertex_index, singularity_index = row
            vertex_index = int(vertex_index)
            b[vertex_index] = K[vertex_index] - 2 * np.pi * singularity_index
        counter = 0
        MAXCOUNTER = 4000
        while len(queue)>0:
            f = queue.popleft()
            vi, vj = f.verts[0].index, f.verts[1].index
            # he = self.mesh.vertex_attributes["hedge"][self.mesh.verts[vi]]
            he = self.mesh.dict_vert2heedges[(vi, vj)]
            keep_looping = True
            holonomy = 0
            sum = 0
            error = 0
            to_add = {}
            sgn_list = {}
            cdebug = 0
            while keep_looping:
                faceI, faceJ = he.face, he.twin.face
                if not has_been_visited[faceJ.index]:
                    u, v = he.vertex, he.twin.vertex
                   
                    e = v.co - u.co
                    sgn = 1
                    # if u.index > v.index:
                    if he.edge.verts[0].index > he.edge.verts[1].index:
                        e = -e
                        sgn = -1
                    sgn = A[vi, he.edge.index]
                    sgn_list[he.edge.index] = sgn
                    ei1, ei2 = self.mesh.face_attributes["e1"][faceI.index], self.mesh.face_attributes["e2"][faceI.index]
                    ej1, ej2 = self.mesh.face_attributes["e1"][faceJ.index], self.mesh.face_attributes["e2"][faceJ.index]
                    delta_ij = np.arctan2(np.dot(ei2, e), np.dot(ei1, e))
                    delta_ji = np.arctan2(np.dot(ej2, e), np.dot(ej1, e))

                    holonomy += delta_ji - delta_ij
                    alpha_i = alphas[faceI.index]
                    sum += sgn * x[he.edge.index]
                    error += - delta_ij + delta_ji - sgn * x[he.edge.index]
                    alpha_j = alpha_i - delta_ij + delta_ji - sgn * x[he.edge.index]
                    alphas[faceJ.index] = alpha_j

                    to_add[faceJ.index] = [alpha_i, -delta_ij + delta_ji, sgn * x[he.edge.index]]

                    vector_field[faceJ.index] = rotate_axis_angle_single_vector(ej1, faceJ.normal, alpha_j)
                    has_been_visited[faceJ.index] = True
                    queue.append(faceJ)
                    cdebug += 1
                    # print(he.edge.index, he.edge.verts[0].index, he.edge.verts[1].index, sgn, A[vi, he.edge.index])
                # if counter==MAXCOUNTER-1:
                #     draw_sphere(u.co, 0.01, "u")
                #     draw_sphere(v.co, 0.01, "v")
                #     w = he.next.next.vertex
                #     cc = (u.co + v.co + w.co)/3
                #     draw_sphere(cc, 0.015, "helper_faceI")
                #     draw_cylinder(he.vertex.co, he.twin.vertex.co, 0.005)
                #     break
                he = he.twin.next
                keep_looping = he != self.mesh.dict_vert2heedges[(vi, vj)]
                # keep_looping = he != self.mesh.vertex_attributes["hedge"][self.mesh.verts[vi]]
                # break
            counter += 1
            while holonomy >= np.pi : holonomy -= 2*np.pi
            while holonomy < -np.pi : holonomy += 2*np.pi

            # if abs(holonomy) - abs(sum) > 1e-7:
            #     raise ValueError(f"incorrect holonomy at vertex {he.vertex.index} : |sum(x)|={abs(sum)}, |holonomy|={holonomy}")
            # elif np.sign(holonomy) == np.sign(sum):
            #     print(f"switching sign at vertex {he.vertex.index}")
            #     for k in to_add:
            #         alpha_i, delta, theta = to_add[k]
            #         to_add[k] = [alpha_i, delta, -theta]


            # for k in to_add:
            #     faceJ = self.mesh.faces[k]
            #     alpha_i, delta, theta = to_add[k]
            #     alpha_j = alpha_i + delta - theta
            #     alphas[faceJ.index] = alpha_j
            #     ej1 = self.mesh.face_attributes["e1"][faceJ.index]

            #     vector_field[faceJ.index] = rotate_axis_angle_single_vector(ej1, faceJ.normal, alpha_j)



            # K = self.mesh.vertex_attributes["angle_defect"]

            
            # # print(A[1985, 3964])
            # idx = np.where(Ad[vi] !=0)[1]
            # print(f"at vertex {vi}, holonomy={holonomy}, sum(x) = {sum}, error={holonomy - sum}")
            # print(y[vi], b[vi],idx, Ad[vi, idx], sgn_list)
            # print(np.where(A.todense()[4]))
            # print(np.sum(x[np.array([   3,   33, 1826, 1973, 3968, 5951])]))
            # if counter == MAXCOUNTER:

            #     break
            
        return vector_field


def rotate_axis_angle_single_vector(vector, axis, angle):
    c = np.cos(angle)
    s = np.sin(angle)
    axis = axis/np.linalg.norm(axis)
    dot = np.dot(axis, vector)
    xprod = np.cross(axis, vector)
    return (1-c) * dot * axis + c * vector + s * xprod


tc = TrivialConnection(bm)
vector_field = tc.compute_vector_field()



if 'vector_field' in mesh.attributes:
    mesh.attributes.remove(mesh.attributes["vector_field"])
    mesh.attributes.remove(mesh.attributes["e1"])


attr = mesh.attributes.new(name="vector_field", type='FLOAT_VECTOR', domain='FACE')
attr.data.foreach_set('vector', vector_field.flatten())

attr = mesh.attributes.new(name="e1", type='FLOAT_VECTOR', domain='FACE')
attr.data.foreach_set('vector', bm.face_attributes["e1"].flatten())

max holonomy error = 1.545430450278218e-13


In [23]:
calculate_holonomy(bm, 1985)

-0.009509551565090368

In [28]:
A = bm._build_d0().T
d1 = bm._build_d1()
# KK = -np.array([calculate_holonomy(bm, i) for i in range(len(bm.verts))])
KK = bm.vertex_attributes["angle_defect"]

b = KK.copy()
b[0] = b[0] - 2*np.pi * 2

AAT = A @ A.T
y = spsolve(AAT, -b)
x = A.T @ y

print(np.max(np.abs(A@x+b)))


2.398081733190338e-13


In [54]:
K

array([0.00540054, 0.00801942, 0.00940796, ..., 0.00484327, 0.00484273,
       0.00950955])

In [55]:
KK

array([-0.00540054, -0.00801942, -0.00940796, ..., -0.00484327,
       -0.00484273, -0.00950955])

In [57]:
np.max(np.abs(KK + K))

3.723128472188364e-10

In [59]:
if 'vector_field' in mesh.attributes:
    mesh.attributes.remove(mesh.attributes["vector_field"])
    mesh.attributes.remove(mesh.attributes["e1"])



attr = mesh.attributes.new(name="vector_field", type='FLOAT_VECTOR', domain='FACE')
attr.data.foreach_set('vector', vector_field.flatten())

attr = mesh.attributes.new(name="e1", type='FLOAT_VECTOR', domain='FACE')
attr.data.foreach_set('vector', bm.face_attributes["e1"].flatten())

In [82]:
mesh.attributes

ReferenceError: StructRNA of type Mesh has been removed